In [140]:
import numpy as np
import pandas as pd
from collections import Counter

"""
Alternative encoder function for pd.getdummies()
"""
def one_hot_encoder(dataf, col):
    mapping = {}
    working_class = dataf[col].unique();

    for i in range(len(working_class)):
        mapping[working_class[i]] = i

    one_hot_code = []
    for w in df[col]:
        temp_arr = list(np.zeros(len(mapping), dtype = int))
        temp_arr[mapping[w]] = 1
        one_hot_code.append(temp_arr)
    df[col] = one_hot_code

In [271]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, gain = 0, y = None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        self.gain = gain
        self.y = y
        self.count = Counter(y)

    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100,  depth=None, n_features=None, criterion = "GINI"):
        self.labels = None
        self.min_samples_splits = min_samples_split
        self.max_depth = max_depth
        self.n_features= n_features
        self.root = None
        self.depth = depth if depth else 0
        self.criterion = criterion

    def fit(self, X, y):
        # Number of features do not exceed the actua feature we have
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.labels = X.columns
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if depth>=self.max_depth or n_labels == 1 or n_samples<self.min_samples_splits:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value, y=y)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        # find the best split
        best_feature, best_thresh, gain = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X.iloc[:, best_feature], best_thresh)
        left = self._grow_tree(X.iloc[left_idxs, :], y.iloc[left_idxs], depth + 1)
        right = self._grow_tree(X.iloc[right_idxs, :], y.iloc[right_idxs], depth + 1)
        return Node(best_feature, best_thresh, left, right, gain, y)

    def _best_split(self, X, y, feat_idxs):
        # initialize best_gain and gini_index
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X.iloc[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                if self.criterion == "GINI":
                    # calculate the GINI gain
                    gain = self.GINI_gain(y, X_column, thr)
                elif self.criterion == "GAIN_RATIO":
                    gain = self._gain_ratio(y, X_column, thr)
                else:
                    # calculate the information gain
                    gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr
        return split_idx, split_threshold, best_gain

    def GINI_gain(self, y, X_column, threshold):
        GINI_base = self.get_GINI(y)

        left_idxs, right_idxs = self._split(X_column, threshold)
        left_counts = Counter(y.iloc[left_idxs])
        right_counts = Counter(y.iloc[right_idxs])

        y0_left, y1_left, y0_right, y1_right = Counter(left_counts).get(0,0),Counter(left_counts).get(1,0), \
                                               Counter(right_counts).get(0,0), Counter(right_counts).get(1,0)

        gini_left = self.GINI_impurity(y0_left, y1_left)
        gini_right = self.GINI_impurity(y0_right, y1_right)

        # Getting the obs count from the left and the right data splits
        n_left = y0_left + y1_left
        n_right = y0_right + y1_right

        # Calculating the weights for each of the nodes
        w_left = n_left / (n_left + n_right)
        w_right = n_right / (n_left + n_right)

        #Calculate the GINI impurity
        wGINI = w_left * gini_left + w_right * gini_right

        #Calculate GINI gain
        GINI_gain = GINI_base - wGINI
        return GINI_gain

    def get_GINI(self, y):
        """
        Calculate the GINI impurity of a node
        """
        y1_count, y2_count = Counter(y).get(0, 0), Counter(y).get(1, 0)
        return self.GINI_impurity(y1_count, y2_count)

    def GINI_impurity(self, y1_count, y2_count):
        if y1_count is None:
            y1_count = 0
        if y2_count is None:
            y2_count = 0
        n = y1_count + y2_count
        # If n is 0 then we return the lowest possible gini impurity
        if n == 0:
            return 0.0
        p1 = y1_count/n
        p2 = y2_count/n
        gini = 1 - (p1**2 + p2**2)
        return gini

    def _gain_ratio(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)
        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y.iloc[left_idxs]), self._entropy(y.iloc[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy

        split_info = -(n_l/n * np.log(n_l/n) +  n_r/n * np.log(n_r/n))

        return information_gain / split_info

    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)
        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y.iloc[left_idxs]), self._entropy(y.iloc[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column.to_numpy() <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column.to_numpy() > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])

    def _most_common_label(self, y):
        counter = Counter(y)
        if len(Counter(y).most_common(1)) == 0:
            value = None
        else:
            value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for idx, x in X.iterrows()])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node() or node.feature is None:
            return node.value
        if x[node.feature] <= node.threshold :
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def counter_to_str(self, counter: Counter):
        inside = [f"{key}: {value}" for key, value in counter.items()]
        return f"({', '.join(inside)})"

    def print_info(self, node, depth):
        """
        Method to print the information about the tree
        """
        if node.feature is None:
            return

        preamble0 = f"|{'---' * 3 * depth}"
        preamble1 = f"|{'   ' * 3 * depth}   |"

        print(f"{preamble0} Split rule: {self.labels[node.feature]} <= {node.threshold}")
        print(f"{preamble1} Gain info the node {round(node.left.gain, 5)}")
        print(f"{preamble1} Class distribution in the node {self.counter_to_str(node.left.count)}")
        # print(f"{preamble1} Predicted class {node.predict}")

        self.print_info(node.left, depth + 1)

        print(f"{preamble0} Split rule: {self.labels[node.feature]} > {node.threshold}")
        print(f"{preamble1} Gain info of the node {round(node.right.gain, 5)}")
        print(f"{preamble1} Class distribution in the node {self.counter_to_str(node.right.count)}")
        # print(f"{preamble1} Predicted class {node.predict}")

        self.print_info(node.right, depth + 1)

    def print_tree(self):
        """
        Prints the whole tree from the current node to the bottom
        """
        if self.root.is_leaf_node():
            return
        else:
            self.print_info(self.root, 1)


In [272]:
# Read in data file
df = pd.read_csv('adult.data', nrows=300, sep=", ", engine='python')
columns_names = ['age', 'workclass', 'fnlwgt',
                 'education', 'education-num', 'marital-status', 'occupation',
                 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
                 'hours-per-week', 'native-country', 'income']
df.columns = columns_names
# Check and transfer columns that are categorical
for col in columns_names[:-1]:
    if isinstance(df[col][0], str):
        df[col] = df[col].astype("category")

df.dropna()
"""
Preprocessing dataframe by encoding categorical data type column into 0 and 1
"""
data = pd.DataFrame()
for col in columns_names[:-1]:
    if df[col].dtype == "category":
        one_hot = pd.get_dummies(df[col], prefix=col)
        # for c in one_hot.columns:
        #     data[c] = one_hot[c].copy()
        data = data.join(one_hot)
    else:
        data[col] = df[col]


data['income'] = df['income']
data.income.replace(('<=50K', '>50K'), (1, 0), inplace=True)

# Attributes of class income to split
train = data.sample(frac=0.6, random_state= 213)

# Class income
test = data.drop(train.index)

X_train = train[train.columns[:-1]]
X_test = test[test.columns[:-1]]
y_train = train['income']
y_test = test['income']

#
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.4, random_state=41347
# )

clf = DecisionTree(max_depth=7, criterion="GINI")
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
predictions_tr = clf.predict(X_train)

print("Root")
clf.print_tree()

# Check accuracy of the Decision Tree
def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)
acc = accuracy(y_test, predictions)

acc_train = accuracy(y_train, predictions_tr)
print(acc_train, acc)

Root
|--------- Split rule: marital-status_Married-civ-spouse <= 0
|            | Gain info the node 0.01901
|            | Class distribution in the node (1: 97, 0: 3)
|------------------ Split rule: education-num <= 14
|                     | Gain info the node 0.00445
|                     | Class distribution in the node (1: 97, 0: 2)
|--------------------------- Split rule: education_Assoc-voc <= 0
|                              | Gain info the node 0.00241
|                              | Class distribution in the node (1: 94, 0: 1)
|------------------------------------ Split rule: workclass_? <= 0
|                                       | Gain info the node 0
|                                       | Class distribution in the node (1: 87)
|------------------------------------ Split rule: workclass_? > 0
|                                       | Gain info of the node 0.21875
|                                       | Class distribution in the node (1: 7, 0: 1)
|-------------------